<a href="https://colab.research.google.com/github/noble752/nsysuhomework/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 10.9 MB/s eta 0:00:00


In [7]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

def ngrok_start():
    ngrok.set_auth_token(userdata.get('ngrok'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    ImageMessage,
    StickerMessage,
    VideoMessage,
    LocationMessage


)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('Channel_access_token'))
handler = WebhookHandler(userdata.get('Channel_secret'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

import google.generativeai as genai
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel("gemini-2.0-flash-exp")
def ask_gemini(question):
    response = model.generate_content(question)
    return response.text

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        action = event.message.text
        if action == "sticker":
          reply = StickerMessage(
            package_id='446',
            stickr_id='1989'
         )
        elif action =='image':
          reply = ImageMessage(
            original_content_url='https://media.istockphoto.com/id/2168400904/photo/colorful-danxia-landform.jpg?s=2048x2048&w=is&k=20&c=WiOJp70CeAjSPLe3TrWIcitRo_XQlOFZrJgkTiGC1gw=',
            preview_image_url='https://media.istockphoto.com/id/2168400904/photo/colorful-danxia-landform.jpg?s=2048x2048&w=is&k=20&c=WiOJp70CeAjSPLe3TrWIcitRo_XQlOFZrJgkTiGC1gw='
         )
        elif action =='video':
          reply = VideoMessage(
            original_content_url='https://media.istockphoto.com/id/2172474150/video/close-up-f16-flying-sunset.mp4?s=mp4-640x640-is&k=20&c=H2VXhCU2Re3qBCtDIWwmbo6Tqi9TK6-KNEXuhTz337k=',
            preview_image_url='https://media.istockphoto.com/id/2172474150/video/close-up-f16-flying-sunset.mp4?s=mp4-640x640-is&k=20&c=H2VXhCU2Re3qBCtDIWwmbo6Tqi9TK6-KNEXuhTz337k='
        )
        elif action =='location':
          reply = LocationMessage(
            title='Taipei 101',
            address='110台北市信義區信義路五段7號',
            latitude=25.034112070859265,
            longitude=121.56451207398551
        )
        else:
         response = ask_gemini(action)
         reply = TextMessage(text=response)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                   reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://e55c-35-196-245-229.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:32:26] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:32:27] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:32:34] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:32:47] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:32:56] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:33:15] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:33:22] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:34:48] "POST /callback HTTP/1.1" 200 -
